In [1]:
# import packages
import geopandas as gpd
import osmnx as ox
import momepy
import pandana as pdna
import pandas as pd
import numpy as np
import networkx as nx

from scipy.spatial import cKDTree
from shapely.geometry import Point

from shapely.geometry import box

In [2]:
## load data

# load geopackages
lsoas = gpd.read_file(r'C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\Centriods\YorkLSOAsSinglePartGPKG.gpkg')


# get location
place = "York, United Kingdom"
walk_dist = 1200 # distance to be walked
no_pois = 700 # max number of points to look for
type = 'walk'  # network type to get. walk, bike, all or drive

cities = ox.geocode_to_gdf([place])

# get all amenities for a given study area

tags = {"amenity": ["bar","cafe","pub","restaurant",
                    "college","kindergarten","library","school","university", "childcare",
                    "bicycle_parking","bicycle_repair_station","bicycle_rental","bus_station","ferry_terminal",
                    "taxi","atm","bank","bureau_de_change",
                    "clinic","dentist","doctors","hospital","pharmacy","social_facility","veterinary"
                    ,"arts_centre","cinema","community_centre","public_bookcase", "kitchen",
                    "social_centre", "theatre", "marketplace", "place_of_worship",
                    "police", "fire_station","post_box", "post_depot", "post_office", "townhall",
                    "drinking_water","toilets","water_point", "parcel_locker", "shower", "telephone",
                    "recycling"]}

pois = ox.geometries_from_place(place, tags)

# load streets from OSM
walk_graph = ox.graph_from_place(place, network_type = type) # download walking network

# get some barriers
barriers = gpd.read_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\Barriers\YorkBarriers.shp")


c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


In [3]:
## clean data

# streets
walk_graph = ox.get_undirected(walk_graph) # cleans the network keeping parallel edges only if geometry is different
walk_graph = ox.projection.project_graph(walk_graph) # project graph
walk_streets = ox.graph_to_gdfs(walk_graph, nodes=False, edges=True, node_geometry=False, fill_edge_geometry=True) # convert graph to gdf
walk_streets = walk_streets[["geometry", "from", "to", "length"]] # clean columns
walk_streets = walk_streets.to_crs(27700) # set crs

In [4]:
# lsoas
lsoas = lsoas.reset_index(drop=True).explode().reset_index(drop=True) # clean
lsoas['lsoaID'] = range(0,len(lsoas)) # generate index column
lsoas = lsoas.set_index('lsoaID') # set index
lsoas = lsoas.to_crs(27700) # set crs

C:\Users\b8008458\AppData\Local\Temp\ipykernel_5780\2443919015.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  lsoas = lsoas.reset_index(drop=True).explode().reset_index(drop=True) # clean


In [5]:
# pois

pois = pois.reset_index(drop=True).explode().reset_index(drop=True) # avoid multipart pois
pois['poiID'] = range(0,len(pois))
pois = pois.set_index('poiID')

# convert polygons to points
pois['geometry'] = pois.centroid

# clean columns
pois = pois[['geometry','amenity']]
pois = pois.to_crs(27700) # set crs

C:\Users\b8008458\AppData\Local\Temp\ipykernel_5780\598720326.py:3: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  pois = pois.reset_index(drop=True).explode().reset_index(drop=True) # avoid multipart pois
C:\Users\b8008458\AppData\Local\Temp\ipykernel_5780\598720326.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pois['geometry'] = pois.centroid


In [6]:
pois.to_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\pois.gpkg")

In [7]:
## generate pandana network
# nodes and edges for walk network
nodes_walk, edges_walk = momepy.nx_to_gdf( # convert network to gdf
    momepy.gdf_to_nx( # convert to nx graph
        walk_streets.explode() # remove multipart rows
    )
)
nodes_walk = nodes_walk.set_index('nodeID') # set index

# generate walk pandana network
walk_streets_pdna = pdna.Network( 
    nodes_walk.geometry.x,
    nodes_walk.geometry.y,
    edges_walk['node_start'], # set origins
    edges_walk['node_end'], # set destinations
    edges_walk[['mm_len']] # set edge length
)

C:\Users\b8008458\AppData\Local\Temp\ipykernel_5780\1581756382.py:5: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  walk_streets.explode() # remove multipart rows


In [8]:
# attach pois to the network
walk_streets_pdna.set_pois( # snap pois to network
    category = 'pois', # set name of the new layer snapped on the network
    maxdist = walk_dist, # set maximum distance
    maxitems = no_pois, # set maximum number of pois to look for
    x_col = pois.geometry.x,
    y_col = pois.geometry.y
)
results = walk_streets_pdna.nearest_pois( # calculate distances to pois
    distance = walk_dist, # maximum distance
    category = 'pois', # layer where we want to look for
    num_pois = no_pois, # max number of pois to look for
    include_poi_ids = True # include pois ids
)

c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\pandana\network.py:660: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  elif isinstance(maxitems, type(pd.Series())):
c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\pandana\network.py:668: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  elif isinstance(maxdist, type(pd.Series())):


In [9]:
# store results separately as distances and poiIDs

# separate distances from poi ids
distances = results.iloc[:,:round(len(results.columns)/2,)] # create df with distances
pois_ids = results.iloc[:,round(len(results.columns)/2,):] # create df with pois ids

# convert wide matrices to long
distances_long = pd.melt(distances.reset_index(), id_vars='nodeID',value_name='distance') # make matrix long
pois_ids_long = pd.melt(pois_ids.reset_index(), id_vars='nodeID',value_name='poiID') # make matrix long

# create an od long df containing nodeID, distance, and poiID
od = distances_long
od['poiID'] = pois_ids_long['poiID'].astype('Int64') # set a column with pois ids (as they are indexed, they are already in the right order)

# format od matrix and drop NAs
od = od[['nodeID','poiID','distance']] # clean columns
od = od.dropna() # drop NAs

In [10]:
# merge od data with POIs data
pois = pois.reset_index() # reset index pois df


In [11]:
od_pois_info = pd.merge(od, pois[['amenity','poiID']].reset_index(), left_on='poiID', right_on='poiID') # merge pois info to od matrix

In [12]:
# add lsoa information to the od_pois_info 
lsoa_nodes = walk_streets_pdna.get_node_ids( # get nearest street nodes to each postcode
    lsoas.geometry.x,
    lsoas.geometry.y
)
lsoa_nodes = gpd.GeoDataFrame(lsoa_nodes).reset_index() # reset index
lsoa_nodes = lsoa_nodes.rename(columns={'node_id':'nodeID'}) # change col names



In [13]:
# get lsoa geometries
lsoa_nodes = lsoa_nodes.merge(lsoas, on='lsoaID', how='left', suffixes=('','_y'))
lsoa_nodes.drop(lsoa_nodes.filter(regex='_y$').columns, axis =1, inplace=True)
lsoa_nodes = lsoa_nodes[['lsoaID','nodeID','geometry']]

In [14]:
# add lsoas to od_pois_info
od_pois_info = pd.merge(od_pois_info, lsoa_nodes, left_on='nodeID', right_on='nodeID') # add lsoa to od matrix

# to csv file 
od_pois_info.to_csv(r'C:\Users\b8008458\Documents\2021_2022\Scratch Space\od_pois_info.csv', index=False)

In [15]:
# save to geospatail format
od_pois_info = gpd.GeoDataFrame(od_pois_info, geometry = 'geometry')
od_pois_info.to_file(r'C:\Users\b8008458\Documents\2021_2022\Scratch Space\od_pois_info.shp', index=False)

In [16]:
# count the number of aminites accessable from a given lsoa
lsoa_level_accessablity = od_pois_info.groupby("lsoaID").size()

In [17]:
# convert to dataframe
lsoa_level_accessablity = pd.DataFrame(lsoa_level_accessablity)

# merge with orignal lsoa file
lsoas = lsoas.merge(lsoa_level_accessablity, on='lsoaID', how='left', suffixes=('','_y'))
lsoas.drop(lsoas.filter(regex='_y$').columns, axis =1, inplace=True)
lsoas.rename(columns = {0:'amenity count'}, inplace = True)

In [18]:
# output to file
lsoas.to_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\Accessability Measures\YorkAccessNoBarriersBike.gpkg")

Now repeat process with a broken network

In [19]:
# create a network with edges with barriers on removed to simulate not being able to pass through a barrier
G = ox.graph_from_place(place, simplify=True, network_type=type)
G_edges = ox.graph_to_gdfs(ox.get_undirected(G), nodes=False, edges=True, node_geometry=False, fill_edge_geometry=True)
G_nodes = ox.graph_to_gdfs(ox.get_undirected(G), nodes=True, edges=False, node_geometry=True, fill_edge_geometry=False)
shply_line = G_edges.geometry.unary_union 
point = barriers.to_crs(G_edges.crs)
for i in range(len(point)):
    print(shply_line.interpolate(shply_line.project( point.geometry[i])).wkt) # snap points to line
result = point.copy()
result['geometry'] = result.apply(lambda row: shply_line.interpolate(shply_line.project( row.geometry)), axis=1)
buffer = result.geometry.buffer(0.00001) # create tiny buffer around points
broken_network = G_edges.intersects(buffer.unary_union)
broken_network = pd.DataFrame(broken_network)
broken_network.rename(columns = {0:'Clipped'}, inplace = True) # clip network
broken_network = pd.concat([broken_network,G_edges],axis=1)
# drop intersected lines
broken_network.drop(broken_network[broken_network['Clipped'] == True].index, inplace=True)
broken_network = broken_network[['geometry','to','from']]
broken_network = gpd.GeoDataFrame(broken_network, geometry='geometry')
G_edges = broken_network.to_crs(3857)
G_edges['length'] = G_edges.length
G_edges = G_edges.to_crs(4326)
G_edges = G_edges.to_crs(27700)
# G = ox.graph_from_gdfs(G_nodes, G_edges)

#G_after_processing = ox.graph_from_gdfs(G_nodes, G_edges)
#G_after_processing2 = momepy.nx_to_gdf(G_after_processing)
#print(G_after_processing)

POINT (-1.083041216634224 53.94557926654369)
POINT (-1.0739283 53.9441324)
POINT (-1.0739283 53.9441324)
POINT (-1.0739283 53.9441324)
POINT (-1.0797802 53.9450607)
POINT (-1.0738031 53.9451816)
POINT (-1.0732171 53.9470538)
POINT (-1.066155850653157 53.946130392666426)
POINT (-1.059554666188326 53.947470729338484)
POINT (-1.058435955680703 53.949930480875096)
POINT (-1.0534923043921502 53.94992674753619)
POINT (-1.048532460812328 53.94932347191377)
POINT (-1.0470913631995158 53.95011685417154)
POINT (-1.0449237655577528 53.950045630109955)
POINT (-1.045820272112785 53.95360492149316)
POINT (-1.0506803074754514 53.95973791439855)
POINT (-1.04955 53.96077)
POINT (-1.0458818 53.9593264)
POINT (-1.043527672911762 53.960495085014195)
POINT (-1.0383203937939562 53.96134269184238)
POINT (-1.057353891483021 53.95972082289049)
POINT (-1.0389224 54.0334809)
POINT (-1.060401967477918 53.9596528574473)
POINT (-1.0647184 53.9609276)
POINT (-1.0684667 53.9595748)
POINT (-1.073285871801579 53.958339

c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Users\b8008458\AppData\Local\Temp\ipykernel_5780\1022689968.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buffer = result.geometry.buffer(0.00001) # create tiny buffer around points


In [20]:
## clean data

# streets
walk_streets = G_edges[["geometry", "from", "to", "length"]] # clean columns
walk_streets = walk_streets.to_crs(27700) # set crs

In [21]:
## generate pandana network
# nodes and edges for walk network
nodes_walk, edges_walk = momepy.nx_to_gdf( # convert network to gdf
    momepy.gdf_to_nx( # convert to nx graph
        walk_streets.explode() # remove multipart rows
    )
)
nodes_walk = nodes_walk.set_index('nodeID') # set index

# generate walk pandana network
walk_streets_pdna = pdna.Network( 
    nodes_walk.geometry.x,
    nodes_walk.geometry.y,
    edges_walk['node_start'], # set origins
    edges_walk['node_end'], # set destinations
    edges_walk[['mm_len']] # set edge length
)

C:\Users\b8008458\AppData\Local\Temp\ipykernel_5780\1581756382.py:5: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  walk_streets.explode() # remove multipart rows


In [22]:
# attach pois to the network
walk_streets_pdna.set_pois( # snap pois to network
    category = 'pois', # set name of the new layer snapped on the network
    maxdist = walk_dist, # set maximum distance
    maxitems = no_pois, # set maximum number of pois to look for
    x_col = pois.geometry.x,
    y_col = pois.geometry.y
)
results = walk_streets_pdna.nearest_pois( # calculate distances to pois
    distance = walk_dist, # maximum distance
    category = 'pois', # layer where we want to look for
    num_pois = no_pois, # max number of pois to look for
    include_poi_ids = True # include pois ids
)

c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\pandana\network.py:660: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  elif isinstance(maxitems, type(pd.Series())):
c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\pandana\network.py:668: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  elif isinstance(maxdist, type(pd.Series())):


In [23]:
# store results separately as distances and poiIDs

# separate distances from poi ids
distances = results.iloc[:,:round(len(results.columns)/2,)] # create df with distances
pois_ids = results.iloc[:,round(len(results.columns)/2,):] # create df with pois ids

# convert wide matrices to long
distances_long = pd.melt(distances.reset_index(), id_vars='nodeID',value_name='distance') # make matrix long
pois_ids_long = pd.melt(pois_ids.reset_index(), id_vars='nodeID',value_name='poiID') # make matrix long

# create an od long df containing nodeID, distance, and poiID
od = distances_long
od['poiID'] = pois_ids_long['poiID'].astype('Int64') # set a column with pois ids (as they are indexed, they are already in the right order)

# format od matrix and drop NAs
od = od[['nodeID','poiID','distance']] # clean columns
od = od.dropna() # drop NAs

In [24]:
# merge od data with POIs data
pois = pois.reset_index() # reset index pois df

In [25]:
od_pois_info = pd.merge(od, pois[['amenity','poiID']].reset_index(), left_on='poiID', right_on='poiID') # merge pois info to od matrix

In [26]:
# add lsoa information to the od_pois_info 
lsoa_nodes = walk_streets_pdna.get_node_ids( # get nearest street nodes to each postcode
    lsoas.geometry.x,
    lsoas.geometry.y
)
lsoa_nodes = gpd.GeoDataFrame(lsoa_nodes).reset_index() # reset index
lsoa_nodes = lsoa_nodes.rename(columns={'node_id':'nodeID'}) # change col names


In [27]:
# get lsoa geometries
lsoa_nodes = lsoa_nodes.merge(lsoas, on='lsoaID', how='left', suffixes=('','_y'))
lsoa_nodes.drop(lsoa_nodes.filter(regex='_y$').columns, axis =1, inplace=True)
lsoa_nodes = lsoa_nodes[['lsoaID','nodeID','geometry']]

In [28]:
# add lsoas to od_pois_info
od_pois_info = pd.merge(od_pois_info, lsoa_nodes, left_on='nodeID', right_on='nodeID') # add lsoa to od matrix

# to csv file 
od_pois_info.to_csv(r'C:\Users\b8008458\Documents\2021_2022\Scratch Space\od_pois_info_barrier.csv', index=False)

In [29]:
# save to geospatail format
od_pois_info = gpd.GeoDataFrame(od_pois_info, geometry = 'geometry')
od_pois_info.to_file(r'C:\Users\b8008458\Documents\2021_2022\Scratch Space\od_pois_info_barrier.shp', index=False)

In [30]:
# count the number of aminites accessable from a given lsoa
lsoa_level_accessablity = od_pois_info.groupby("lsoaID").size()

In [31]:
# convert to dataframe
lsoa_level_accessablity = pd.DataFrame(lsoa_level_accessablity)

# merge with orignal lsoa file
lsoas = lsoas.merge(lsoa_level_accessablity, on='lsoaID', how='left', suffixes=('','_y'))
lsoas.drop(lsoas.filter(regex='_y$').columns, axis =1, inplace=True)
lsoas.rename(columns = {0:'amenity count barriers'}, inplace = True)

In [32]:
lsoas['amenity_diff'] =  lsoas['amenity count'].sub(lsoas['amenity count barriers'], axis = 0)

In [33]:
# output to file
lsoas.to_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\Accessability Measures\YorkAccessBarriersBike.gpkg")

In [34]:
pois.to_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\pois.gpkg")

Calculate isochrones around each barrier

In [59]:
# function to get isochrones
def get_isochrone(
    lon, lat, walk_times=[60, 120], speed=24, name=None, point_index=None
):
    loc = (lat, lon)
    G = ox.graph_from_point(loc, dist=10000, dist_type='bbox', simplify=True, network_type="all")
    gdf_nodes = ox.graph_to_gdfs(G, edges=False)
    center_node = ox.distance.nearest_nodes(G, lon, lat)
    #fig, ax = ox.plot_graph(G,node_color="w", node_size= 50)

    meters_per_minute = speed * 1000 / 60  # km per hour to m per minute
    for u, v, k, data in G.edges(data=True, keys=True):
        data["time"] = data["length"] / meters_per_minute
    polys = []
    for walk_time in walk_times:
        subgraph = nx.ego_graph(G, center_node, radius=walk_time, distance="time")
        node_points = [
            Point(data["x"], data["y"]) for node, data in subgraph.nodes(data=True)
        ]
        polys.append(gpd.GeoSeries(node_points).unary_union.convex_hull)
    info = {}
    if name:
        info["name"] = [name for t in walk_times]
    if point_index:
        info["point_index"] = [point_index for t in walk_times]
    return {**{"geometry": polys, "time": walk_times}, **info}


In [60]:
# calculate isochrones

WT = [5, 10, 15]
BARRIERS = len(barriers)

# build geopandas data frame of isochrone polygons for each barrier
isochrones = pd.concat(
    [
        gpd.GeoDataFrame(
            get_isochrone(
                r["geometry"].x,
                r["geometry"].y,
                name=r["globalid"],
                point_index=i,
                walk_times=WT,
            ),
            crs=barriers.crs,
        )
        for i, r in barriers.head(BARRIERS).iterrows()
    ]
)


In [61]:
isochrones.head(30)

,geometry,time,name,point_index
0,"POLYGON ((-1.07755 53.92936, -1.09078 53.93013...",5,54ade712-9b6c-4716-96c0-9efb9f62d005,NaN
1,"POLYGON ((-1.09357 53.91307, -1.10675 53.91671...",10,54ade712-9b6c-4716-96c0-9efb9f62d005,NaN
2,"POLYGON ((-1.09158 53.89877, -1.10515 53.89889...",15,54ade712-9b6c-4716-96c0-9efb9f62d005,NaN
0,"POLYGON ((-1.07611 53.92891, -1.09087 53.93083...",5,f6daa07a-d3e7-4f04-abd9-4cae134e562f,1.0
1,"POLYGON ((-1.09364 53.91427, -1.10424 53.91809...",10,f6daa07a-d3e7-4f04-abd9-4cae134e562f,1.0
2,"POLYGON ((-1.09158 53.89877, -1.10515 53.89889...",15,f6daa07a-d3e7-4f04-abd9-4cae134e562f,1.0
0,"POLYGON ((-1.07611 53.92891, -1.09131 53.93412...",5,49d7398c-c98a-4466-903b-6e05a146eed8,2.0
1,"POLYGON ((-1.09364 53.91427, -1.10424 53.91809...",10,49d7398c-c98a-4466-903b-6e05a146eed8,2.0
2,"POLYGON ((-1.09158 53.89877, -1.10667 53.90099...",15,49d7398c-c98a-4466-903b-6e05a146eed8,2.0
0,"POLYGON ((-1.07472 53.92988, -1.09131 53.93412...",5,763d1bb1-363d-4bc6-b6e6-0efdee707937,3.0


In [62]:
# plot isochrones on a map

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=PendingDeprecationWarning)
ox.config(use_cache=True, log_console=False)
warnings.filterwarnings("ignore")

gdf = isochrones.set_index(["time", "point_index"]).copy()
# remove shorter walk time from longer walk time polygon to make folium work better
for idx in range(len(WT)-1,0,-1):
    gdf.loc[WT[idx], "geometry"] = (
        gdf.loc[WT[idx]]
        .apply(
            lambda r: r["geometry"].symmetric_difference(
                gdf.loc[(WT[idx-1], r.name), "geometry"]
            ),
            axis=1,
        )
        .values
    )

m = gdf.reset_index().explore(column="time", scheme="boxplot")
barriers.head(BARRIERS).explore(m=m, marker_kwds={"radius": 3, "color": "red"})


In [1]:
# remove shorter walk distances
isochrones = isochrones[isochrones.time == 15]
isochrones['point_index'] = isochrones['point_index'].fillna(0)
isochrones.head(10)

NameError: name 'isochrones' is not defined

In [2]:
# check crs of isochrones
isochrones.crs

NameError: name 'isochrones' is not defined

In [63]:
# output to file
isochrones.to_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\Accessability Measures\BarrierisochronesBike.gpkg")

In [40]:
# set crs to match isochrones
lsoas = lsoas.to_crs(4326)
lsoas.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [41]:
# join lsoas to isochrones
left_join = lsoas.sjoin(isochrones, how = "left")
barrier_score = left_join.groupby("name").sum("amenity_diff") # calculate accessiablity score per barrier
barrier_score = barrier_score[['objectid', 'amenity_diff']]

In [42]:
barrier_score

,objectid,amenity_diff
name,,
02c77c93-4b8e-40ff-a2a2-7b682d74889f,57938,223
046804fc-dd0d-4f57-86d4-8b1159e3657b,39807,61
0afce902-74cd-4893-b931-53aad32cd1a9,94965,180
0c5cf604-6eee-4527-9a8f-438cb2dd54bc,93877,75
1f1be6c4-7beb-4c15-8cc2-96973ee6e4c1,32024,174
32a88aa0-3563-4813-8f7e-a078af9a15f2,40167,45
332dc7cd-e64d-4685-bb4c-bab1cd855a12,48371,69
3b22a61e-5772-4272-81a4-aa4160ad6492,32024,96
49d7398c-c98a-4466-903b-6e05a146eed8,59357,163


In [43]:
barriers_join = barriers.merge(barrier_score, left_on = 'globalid', right_on = 'name', how='left') # join datafranes
barriers_join['amenity_diff'] = barriers_join['amenity_diff'].fillna(0) # fill nans with 0

In [44]:
barriers_join

,globalid,AuditAmend,CriticalDi,OnNCN,Restrictio,Critical_1,Other,Status,CreationDa,Creator,...,Tranch4,CountAtt_1,PhotoURL,Count_Atta,min_gain,origin_id,total_cost,geometry,objectid,amenity_diff
0,54ade712-9b6c-4716-96c0-9efb9f62d005,Auditing,0,yes,BollardMultiple,105,None,Too Narrow,2022-06-20,RH_Access_Sustrans_UK,...,None,0,None,None,192.430350,54ade712-9b6c-4716-96c0-9efb9f62d005,150.568612,POINT (-1.08308 53.94545),86346,163
1,f6daa07a-d3e7-4f04-abd9-4cae134e562f,Auditing,0,yes,BollardMultiple,224,None,Acceptable Width,2022-06-20,RH_Access_Sustrans_UK,...,None,0,None,None,177.118244,f6daa07a-d3e7-4f04-abd9-4cae134e562f,104.259386,POINT (-1.08148 53.94452),59357,163
2,49d7398c-c98a-4466-903b-6e05a146eed8,Auditing,0,no,BollardSingle,126,None,Not NCN,2022-06-20,RH_Access_Sustrans_UK,...,None,0,None,None,50.991718,49d7398c-c98a-4466-903b-6e05a146eed8,104.499472,POINT (-1.08063 53.94423),59357,163
3,763d1bb1-363d-4bc6-b6e6-0efdee707937,Auditing,0,yes,Chicane,191,None,Acceptable Width,2022-06-20,RH_Access_Sustrans_UK,...,None,0,None,None,35.030644,763d1bb1-363d-4bc6-b6e6-0efdee707937,62.797804,POINT (-1.08071 53.94504),59357,163
4,66f3469b-9eb5-497f-b0c3-ebb2d8c8c076,Auditing,0,yes,BollardMultiple,144,None,Too Narrow,2022-06-20,RH_Access_Sustrans_UK,...,None,0,None,None,183.496445,66f3469b-9eb5-497f-b0c3-ebb2d8c8c076,62.797804,POINT (-1.07982 53.94506),52614,164
5,69970194-9501-4692-8fbb-e0ccbefa9714,Auditing,0,yes,BollardMultiple,151,None,Acceptable Width,2022-06-20,RH_Access_Sustrans_UK,...,None,0,None,None,141.100915,69970194-9501-4692-8fbb-e0ccbefa9714,183.496445,POINT (-1.07291 53.94523),44834,235
6,1f1be6c4-7beb-4c15-8cc2-96973ee6e4c1,Auditing,0,yes,BollardSingle,200,None,Acceptable Width,2022-06-20,RH_Access_Sustrans_UK,...,None,0,None,None,141.100915,1f1be6c4-7beb-4c15-8cc2-96973ee6e4c1,183.496445,POINT (-1.07018 53.94535),32024,174
7,3b22a61e-5772-4272-81a4-aa4160ad6492,Auditing,0,yes,Cattlegrid,130,other,Too Narrow,2022-06-20,RH_Access_Sustrans_UK,...,None,0,None,None,309.720561,3b22a61e-5772-4272-81a4-aa4160ad6492,309.720561,POINT (-1.06616 53.94613),32024,96
8,f03d5d9e-dd6a-4995-84e1-843deef6cd7e,Auditing,0,yes,Cattlegrid,121,other,Too Narrow,2022-06-20,RH_Access_Sustrans_UK,...,None,0,None,None,104.499472,f03d5d9e-dd6a-4995-84e1-843deef6cd7e,298.140497,POINT (-1.05968 53.94752),31371,34
9,866c3440-d9cd-44ff-abef-5aea510dccdd,Auditing,0,yes,BollardMultiple,308,None,Acceptable Width,2022-06-20,RH_Access_Sustrans_UK,...,None,0,None,None,35.030644,866c3440-d9cd-44ff-abef-5aea510dccdd,298.140497,POINT (-1.05846 53.94987),39571,57


In [45]:
barriers_join.to_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\Accessability Measures\YorkAccessBarriersScoredBike.gpkg")